In [2]:
# Pytorch packages
import torch
from torch_geometric.data import Data
import torch.nn as nn
import torch_geometric.nn as pyg_nn
from torch_geometric.utils import to_networkx
from torch_geometric.utils import add_self_loops, degree

# Other packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import networkx as nx
import meshio

In [3]:
# CUDA support
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
class Geometry():
    def __init__(self):
        pass

    def create_graph(self, mesh_file=""):
        self.mesh = meshio.read(mesh_file)
        self.node_pos = torch.tensor(self.mesh.points[:, :2], dtype=torch.float, device=device)

        edges = []

        for cell in self.mesh.cells:
            if cell.type == "triangle":
                for element in cell.data:
                    edges.append([element[0], element[1]])
                    edges.append([element[1], element[0]])

                    edges.append([element[1], element[2]])
                    edges.append([element[2], element[1]])

                    edges.append([element[2], element[0]])
                    edges.append([element[0], element[2]])

        edges = torch.tensor(edges, dtype=torch.long, device=device).t().contiguous()
        source_nodes = edges[0]
        target_nodes = edges[1]
        source_pos = self.node_pos[source_nodes]
        target_pos = self.node_pos[target_nodes]
        edge_length = torch.norm(source_pos - target_pos, dim=1, keepdim=True)
        
        self.graph = Data(x=self.node_pos, edge_index=edges, edge_attr=edge_length).to(device)
        print(self.graph)

    def create_dataset(self):
        pass
